



Ovaj Python kod implementira simulaciju igrice "2048" koristeći pristup backtracking-a kako bi predvidio i ocijenio buduća stanja table i na taj način odredio najbolji potez u svakom koraku. Implementacija je strukturirana oko ključnih funkcija koje upravljaju stanjem igre, izvršavaju akcije igrača i koriste vještačku inteligenciju za simulaciju potencijalnih budućih poteza radi optimizacije strategije.

### Inicijalizacija i postavljanje pločica

- **`initialize_game()`**: Postavlja novu 4x4 igračku tablu kao NumPy niz popunjen nulama, što predstavlja praznu tablu. Inicijalizuje igru postavljanjem dvije pločice na tablu, obično '2', ali ponekad i '4', na osnovu unaprijed definisanih vjerovatnoća.

- **`add_new_tile(board)`**: Identifikuje prazne pozicije na tabli i postavlja novu pločicu (ili '2' ili '4', gde je '4' manje vjerovatna) na slučajno odabrano prazno mesto. Ova funkcija oponaša slučajno postavljanje pločica u stvarnoj igri nakon svakog poteza.

### Funkcije za pomjeranje

Svaka funkcija za pomjeranje (`move_left`, `move_right`, `move_up`, `move_down`) manipuliše tablom tako što klizi pločice u određenom pravcu, kombinuje pločice iste vrijednosti i popunjava nove prazne prostore kako slijedi:

- **`move_left(board)`**: Klizi sve pločice na levo, kombinuje susjedne pločice iste vrednosti u jednu pločicu čija je vrednost njihov zbir, što je osnovni mehanizam igre 2048. Pločice se kombinuju samo jednom po potezu.

- Ostale funkcije za pomeranje su varijacije `move_left`, prilagođene za različite pravce koristeći NumPy funkciju `rot90` za rotiranje table, primjenu funkcije `move_left`, i vraćanje table u originalnu orijentaciju.

### Simulacija i donošenje odluka

- **`simulate_moves(board, depth, path=[])`**: Ova funkcija koristi povratno praćenje za simulaciju svih mogućih poteza od trenutnog stanja table do određene dubine:
  - Za svaki mogući pravac (levo, desno, gore, dole), tabla se kopira i primjenjuje se potez.
  - Ako potez promijeni tablu, dodaje se nova pločica (kao što bi se desilo u stvarnoj igri), i funkcija rekurzivno simulira sledeći potez.
  - Konačna tabla svake simulirane putanje ocjenjuje se koristeći `evaluate_board`, koji uzima u obzir najveću pločicu i broj praznih mesta za određivanje "ocene" table.
  - Putanja sa najvišom ocenom na najdubljem nivou rekurzije smatra se najboljom putanjom za taj niz poteza.

- **`evaluate_board(board)`**: Računa jednostavnu heurističku ocjenu dodavanjem najveće vrednosti pločice na tabli broju praznih mesta. Ova ocjena motiviše AI da postigne veće pločice i održi tablu što praznijom, produžavajući igru.

### Glavna petlja igre

- **`strategic_game_loop()`**: Upravlja napretkom igre ponavljajući simulaciju poteza i primenjujući najbolji potez dok se igra ne završi:
  - Štampa trenutnu tablu.
  - Koristi `simulate_moves` za određivanje najboljeg trenutnog poteza.
  - Primenjuje najbolji potez i dodaje novu pločicu.
  - Provjerava uslove za kraj igre, bilo pobjedom (dosezanjem pločice 2048) ili nemogućnošću daljih poteza.

### Zaključak

Upotreba backtracking-a u ovom skriptu omogućava AI-u da "razmišlja unapred" kroz nekoliko poteza (kako je određeno `depth_of_simulation`), uzimajući u obzir ne samo neposredne posledice poteza, već i kako on postavlja osnovu za buduće mogućnosti. Ovaj pristup pokušava optimizovati svaki potez na osnovu potencijalnih budućih stanja table, čineći ga strateškijim i vjerovatnijim za uspeh u složenom okruženju igre 2048.


In [9]:
import numpy as np
import random

def initialize_game():
    board = np.zeros((4, 4), dtype=int)
    add_new_tile(board)
    add_new_tile(board)
    return board

def add_new_tile(board):
    empty_positions = [(i, j) for i in range(4) for j in range(4) if board[i][j] == 0]
    if empty_positions:
        i, j = random.choice(empty_positions)
        board[i][j] = 4 if random.random() < 0.1 else 2
    return board

def move_left(board):
    new_board = np.zeros((4, 4), dtype=int)
    for i in range(4):
        filtered = board[i][board[i] != 0]
        merged = []
        skip = False
        j = 0
        while j < len(filtered):
            if skip:
                skip = False
                j += 1
                continue
            if j + 1 < len(filtered) and filtered[j] == filtered[j + 1]:
                merged.append(2 * filtered[j])
                skip = True
            else:
                merged.append(filtered[j])
            j += 1
        merged += [0] * (4 - len(merged))
        new_board[i] = merged
    return new_board

def move_right(board):
    rotated_board = np.rot90(board, 2)
    new_board = move_left(rotated_board)
    return np.rot90(new_board, 2)

def move_up(board):
    rotated_board = np.rot90(board, 3)
    new_board = move_left(rotated_board)
    return np.rot90(new_board)

def move_down(board):
    rotated_board = np.rot90(board)
    new_board = move_left(rotated_board)
    return np.rot90(new_board, 3)

def evaluate_board(board):
    max_tile = np.max(board)
    empty_tiles = len(board[board == 0])
    score = max_tile + empty_tiles
    return score

def simulate_moves(board, depth, path=[]):
    if depth == 0:
        return evaluate_board(board), path
    best_score = -1
    best_path = []
    directions = ['left', 'right', 'up', 'down']
    moves = {
        'left': move_left,
        'right': move_right,
        'up': move_up,
        'down': move_down
    }
    for direction in directions:
        new_board = moves[direction](np.copy(board))
        if not np.array_equal(board, new_board):
            new_tile_added = add_new_tile(new_board)
            score, new_path = simulate_moves(new_tile_added, depth - 1, path + [direction])
            if score > best_score:
                best_score = score
                best_path = new_path
    return best_score, best_path

def is_game_won(board):
    return np.any(board == 2048)
def is_game_over(board):
    if np.any(board == 0):
        return False
    for move_func in [move_left, move_right, move_up, move_down]:
        if not np.array_equal(board, move_func(np.copy(board))):
            return False
    return True

def strategic_game_loop():
    board = initialize_game()
    game_over = False
    depth_of_simulation = 7
    moves = {
        'left': move_left,
        'right': move_right,
        'up': move_up,
        'down': move_down
    }
    while not game_over:
        print('Current board:')
        print(board)
        _, best_path = simulate_moves(board, depth_of_simulation)
        if best_path:
            direction = best_path[0]
            print(f'Optimal move: {direction}')
        else:
            direction = random.choice(['left', 'right', 'up', 'down'])
            print(f'Fallback move: {direction}')
        new_board = moves[direction](board)
        if not np.array_equal(board, new_board):
            board = add_new_tile(new_board)
        if is_game_won(board):
            print('Congratulations! You\'ve won the game.')
            print(board)
            game_over = True
        elif is_game_over(board):
            print('Game Over! No more moves available.')
            print(board)
            game_over = True

#testing
strategic_game_loop()

Streaming output truncated to the last 5000 lines.
 [  0   0  32  64]
 [  0   2 128  16]]
Optimal move: right
Current board:
[[  0   0   2   2]
 [  0   2   8   2]
 [  0   0  32  64]
 [  0   2 128  16]]
Optimal move: left
Current board:
[[  4   2   0   0]
 [  2   8   2   0]
 [ 32  64   0   0]
 [  2 128  16   0]]
Optimal move: right
Current board:
[[  0   0   4   2]
 [  0   2   8   2]
 [  0   4  32  64]
 [  0   2 128  16]]
Optimal move: down
Current board:
[[  0   2   4   4]
 [  2   4   8  64]
 [  0   2  32  16]
 [  0   0 128   0]]
Optimal move: right
Current board:
[[  0   2   2   8]
 [  2   4   8  64]
 [  0   2  32  16]
 [  0   0   0 128]]
Optimal move: right
Current board:
[[  0   0   4   8]
 [  2   4   8  64]
 [  2   2  32  16]
 [  0   0   0 128]]
Optimal move: left
Current board:
[[  4   8   2   0]
 [  2   4   8  64]
 [  4  32  16   0]
 [128   0   0   0]]
Optimal move: up
Current board:
[[  4   0   0   0]
 [  2   8   2   2]
 [  4   4   8   0]
 [128  32  16  64]]
Optimal move: right
